<a href="https://colab.research.google.com/github/guptamols/zero_to_hero_nn/blob/main/micrograd_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# micrograd exercises

1. watch the [micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. come back and complete these exercises to level up :)

## section 1: derivatives

In [1]:
# here is a mathematical expression that takes 3 inputs and produces one output
import math

def f(a, b, c):
  return -a**3 + math.sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [2]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):
    da= (-3*a**2) - (0.5*a**-0.5)
    db= 3*math.cos(3*b) + 2.5*b**1.5
    dc= c**-2
    return [da,db,dc] # todo, return [df/da, df/db, df/dc]

# expected answer is the list of
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [3]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

# -----------
numerical_grad = [0, 0, 0] # TODO

def gradf(a, b, c):
    h = 0.000001
    da= (f(a+h,b,c)-f(a,b,c))/h
    db= (f(a,b+h,c)-f(a,b,c))/h
    dc= (f(a,b,c+h)-f(a,b,c))/h
    return [da,db,dc] # todo, return [df/da, df/db, df/dc]

numerical_grad = gradf(2,3,4)

# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353559348809995
OK for dim 1: expected 10.25699027111255, yours returns 10.256991666679482
OK for dim 2: expected 0.0625, yours returns 0.062499984743169534


In [4]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# -----------
numerical_grad2 = [0, 0, 0] # TODO

def gradf(a, b, c):
    h = 0.000001
    da= (f(a+h,b,c)-f(a-h,b,c))/(2*h)
    db= (f(a,b+h,c)-f(a,b-h,c))/(2*h)
    dc= (f(a,b,c+h)-f(a,b,c-h))/(2*h)
    return [da,db,dc] # todo, return [df/da, df/db, df/dc]

numerical_grad2 = gradf(2,3,4)
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553391353245
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027401572
OK for dim 2: expected 0.0625, yours returns 0.06250000028629188


## section 2: support for softmax

In [5]:
# Value class starter code, with many functions taken out

class Value:
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"

    def __add__(self, other): # exactly as in the video
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad

        out._backward = _backward

        return out

  # ------
    def __mul__(self, other): # exactly as in the video
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad

        out._backward = _backward

        return out

    def __pow__(self, other): # exactly as in the video
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data ** other, (self,), f"**{other}")

        def _backward():
            self.grad += (other*self.data**(other-1)) * out.grad

        out._backward = _backward

        return out

    def sin(self):
        out = Value(math.sin(self.data),(self,),label='sin')

        def _backward():
            self.grad += math.cos(self.data) * out.grad
        out._backward = _backward

        return out

    def exp(self):
        out = Value(math.exp(self.data),(self,),label='exp')

        def _backward():
            self.grad += math.exp(self.data) * out.grad
        out._backward = _backward

        return out

    def log(self):
        out = Value(math.log(self.data),(self,),label='log')

        def _backward():
            self.grad += (1/self.data) * out.grad

        out._backward = _backward

        return out

    def __neg__(self): # -self
        return self * -1
  # ------

    def __radd__(self, other): # other + self
        return self + other

    def __sub__(self, other): # self - other
        return self + (-other)

    def __rsub__(self, other): # other - self
        return other + (-self)

    def __rmul__(self, other): # other * self
        return self * other

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __rtruediv__(self, other): # other / self
        return other * self**-1

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"

    def backward(self): # exactly as in video

        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)

        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

In [6]:
a=Value(2.0,label='a')
b=Value(3.0,label='b')
c=Value(4.0,label='c')

In [7]:
o = -a**3 + (3*b).sin() - 1.0/c + b**2.5 - a**0.5

In [8]:
o.backward()

In [9]:
print(a.data, b.data, c.data)

2.0 3.0 4.0


In [10]:
a,b,c

(Value(data=2.0, grad=-12.353553390593273),
 Value(data=3.0, grad=10.25699027111255),
 Value(data=4.0, grad=0.0625))

In [11]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


In [12]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch
import torch.nn.functional as F

In [13]:
logits = torch.tensor([0.0, 3.0, -2.0, 1.0], requires_grad=True)
probs = F.softmax(logits, dim=0)

In [14]:
probs

tensor([0.0418, 0.8390, 0.0057, 0.1135], grad_fn=<SoftmaxBackward0>)

In [15]:
loss = -probs[3].log()
loss

tensor(2.1755, grad_fn=<NegBackward0>)

In [16]:
loss.backward()

In [17]:
print('Loss:', loss.item())
print('Gradients:', logits.grad)

Loss: 2.1755154132843018
Gradients: tensor([ 0.0418,  0.8390,  0.0057, -0.8865])
